In [14]:
# For the data of this project go to: 'https://www.kaggle.com/datasets/paololol/league-of-legends-ranked-matches/data'
#!pip install ipython-sql

In [15]:
import pandas as pd
import sqlite3
%load_ext sql 

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [16]:
%sql sqlite:///lolanalysis.db
conn =  sqlite3.connect('lolanalysis.db')

<b> Create table with prefered column in the sql database <b>

In [17]:
%%sql
drop table if exists Participants;
drop table if exists Champs;
drop table if exists Stats;

create table Participants (
    id int,
    matchid int,
    player int,
    championid int,
    position varchar(8)
);
create table Champs ( 
    name varchar(16),
    id int
);
create table Stats (
    id int,
    win int
)

 * sqlite:///lolanalysis.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

<b> Read excel files, clear unnecessary columns and transfer to sql <b>

In [18]:
# There are 3 files named Participants, Stat1 and Stat2 have more than 1 million records so we need to make a function
# to insert values into sql since the max entries per insert is 999 

def InsertValue(df_name , sql_table):
    # Chunk size for inserting data into SQL database
    chunksize = 1000
    # Define the number of chunks
    num_chunks = len(df_name) // chunksize + 1
    # Insert data into SQL database in chunks
    for i in range(num_chunks):
        start_idx = i * chunksize
        end_idx = (i + 1) * chunksize
        chunk_df = df_name.iloc[start_idx:end_idx]
        chunk_df.to_sql(sql_table, conn, if_exists='append', index=False)



participant_df = pd.read_csv(r"C:\Users\VINH\Desktop\lol analyst\participants.csv")


# convert player to two sides to indentify enemies and allies
for i in range(1,6): 
    participant_df['player'] = participant_df['player'].replace( to_replace = i, value = 0)
for i in range(6,11):
    participant_df['player'] = participant_df['player'].replace( to_replace= i, value = 1)

# merge 2 columns: role and position
participant_df['merged'] = participant_df['role'] + ' ' + participant_df['position']

# drop unnecessary columns
participant_df.drop(columns=['ss1','ss2','role','position'], inplace=True)

participant_df.rename(columns= {'merged':'position'}, inplace=True)
participant_df['position'] = participant_df['position'].replace( to_replace = ['SOLO TOP', 'SOLO MID', 'NONE JUNGLE', 'DUO_SUPPORT BOT','DUO_CARRY BOT'], 
                                value= ['Top', 'Mid', 'Jungle', 'Support', 'Adc'] )
#  re-check the df 
print(participant_df.head(5))
# transfer the df to sql:  participant_df.to_sql('Participants', conn, if_exists= 'replace' , method = 'multi')  ( this doesn't work because of the numbers of variables exceed the capacity  of sqlite)
InsertValue(participant_df, 'Participants')


champ_df = pd.read_csv(r"C:\Users\VINH\Desktop\lol analyst\champs.csv")
print(champ_df.head(5))
champ_sql = champ_df.to_sql('Champs', conn, if_exists= 'replace', method='multi')

winstat1_df = pd.read_csv(r"C:\Users\VINH\Desktop\lol analyst\stats1.csv")
winstat1_df = winstat1_df[['id','win']]
print(winstat1_df.head(5))
InsertValue(winstat1_df, 'Stats')

winstat2_df = pd.read_csv(r"C:\Users\VINH\Desktop\lol analyst\stats2.csv")
winstat2_df = winstat2_df[['id','win']]
print(winstat2_df.head(5))
InsertValue(winstat2_df, 'Stats')


   id  matchid  player  championid position
0   9       10       0          19   Jungle
1  10       10       0         267  Support
2  11       10       0         119      Adc
3  12       10       0         114      Top
4  13       10       0         112      Mid
       name   id
0       Jax   24
1      Sona   37
2  Tristana   18
3     Varus  110
4     Fiora  114
   id  win
0   9    0
1  10    0
2  11    0
3  12    0
4  13    0


C:\Users\VINH\AppData\Local\Temp\ipykernel_2424\1863289335.py:51: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  winstat2_df = pd.read_csv(r"C:\Users\VINH\Desktop\lol analyst\stats2.csv")


        id  win
0  1028382    0
1  1028383    0
2  1028384    0
3  1028385    0
4  1028386    0
